## QUBO warm start for the battery MILP (24-qubit discharge-timing skeleton)

### Goal
We want a small QUBO (≤ 24 binary variables) that provides an initial guess for the discharge profile $d_t$, which can then be refined by a classical LP/MILP that enforces SOC dynamics and all engineering constraints.

Because of the no netting rule, the wind-scenario revenue term is constant w.r.t. battery decisions, so the battery schedule is driven by prices $p_t$ (and battery constraints). The QUBO therefore uses only $p_t$.

---

## Step 1 — Skeleton decision variables (24 qubits)

Define one binary variable per hour: $x_t \in \{0,1\}$ for $t=1,\dots,24$.

Interpretation:
- $x_t=1$ means “discharge during hour $t$”
- $x_t=0$ means “do not discharge during hour $t$”

To keep the QUBO minimal, we use a coarse discharge magnitude:
$\tilde d_t := 4x_t$ (MWh).

This yields a binary timing skeleton for discharge, which is the warm-start signal we want.

---

## Step 2 — Revenue term in QUBO form

Battery discharge revenue under the skeleton approximation is:
$\sum_{t=1}^{24} p_t \tilde d_t = \sum_{t=1}^{24} 4p_t x_t$.

A QUBO is typically a minimization, so we minimize negative revenue:
$E_{\mathrm{rev}}(x) := -\sum_{t=1}^{24} 4p_t x_t$.

---

## Step 3 — Continuity via a switching penalty

To discourage fragmented on/off behavior, add a quadratic penalty on switches:
$E_{\mathrm{sw}}(x) := \gamma \sum_{t=2}^{24} (x_t-x_{t-1})^2$, with $\gamma\ge 0$.

For binary variables, $x_t^2=x_t$, hence:
$(x_t-x_{t-1})^2 = x_t + x_{t-1} - 2x_tx_{t-1}$.

Interpretation:
- $\gamma$ acts like a cost per switch (up to scaling)
- larger $\gamma$ encourages fewer discharge blocks

---

## Step 4 — Cycle proxy as a cardinality penalty (no extra qubits)

The MILP cycle proxy is:
$\sum_{t=1}^{24} d_t \le 32$ MWh.

Under $\tilde d_t=4x_t$, this becomes:
$4\sum_{t=1}^{24} x_t \le 32$, i.e. $\sum_{t=1}^{24} x_t \le 8$.

With only 24 qubits, a standard way to enforce this without slack bits is a target-cardinality penalty:
$E_{\mathrm{card}}(x) := A\left(\sum_{t=1}^{24} x_t - K\right)^2$, with $A>0$ and $K=8$.

Notes:
- This enforces $\sum_t x_t \approx K$ (typically $\sum_t x_t = K$ if $A$ is large).
- This is a modification of the original inequality, acceptable for warm-starting because the classical refinement step will enforce SOC physics and may reshape discharges.

---

## Step 5 — Final QUBO objective

The final QUBO is:
$\min_{x\in\{0,1\}^{24}} E(x)$

with:
$E(x)=E_{\mathrm{rev}}(x)+E_{\mathrm{sw}}(x)+E_{\mathrm{card}}(x)$

i.e.:
$E(x)=-\sum_{t=1}^{24}4p_t x_t + \gamma\sum_{t=2}^{24}(x_t-x_{t-1})^2 + A\left(\sum_{t=1}^{24}x_t-K\right)^2$.

This uses exactly 24 binary variables.

---

## Step 6 — QUBO matrix form $x^\top Q x$

Write $x=(x_1,\dots,x_{24})^\top$. A QUBO can be written as:
$E(x)=x^\top Q x$,

using the convention:
$x^\top Q x = \sum_t Q_{tt}x_t + \sum_{i<j} Q_{ij}x_ix_j$.

### Diagonal terms (linear coefficients)

Define:
$\deg(t)=1$ for $t\in\{1,24\}$, and $\deg(t)=2$ for $t\in\{2,\dots,23\}$.

Then:
$Q_{tt} = -4p_t + \gamma\,\deg(t) + A(1-2K)$.

### Off-diagonal terms (quadratic couplers)

Cardinality penalty contributes for all $i<j$:
$Q_{ij} \mathrel{+}= 2A$.

Switching penalty contributes for adjacent hours:
$Q_{t-1,t} \mathrel{+}= -2\gamma$ for $t=2,\dots,24$.

Equivalently:
- if $|i-j|=1$, then $Q_{ij}=2A-2\gamma$
- if $|i-j|>1$, then $Q_{ij}=2A$

(Additive constants such as $AK^2$ can be dropped since they do not affect the minimizer.)

---

## Step 7 — Warm start for the MILP

After solving the QUBO, decode the binary solution $x_t^\star$ and define:
$d_t^{(0)} := 4x_t^\star$.

Then run a classical projection/refinement that enforces the real battery physics:
- SOC dynamics: $E_t = E_{t-1} + \eta_{\mathrm{ch}}c_t - d_t$, with $E_0=E_{24}=0$ and $0\le E_t\le 16$
- power limits: $0\le c_t\le 5$ and $0\le d_t\le 4$
- optionally: no-reversal and/or block penalties

A typical refinement approach is:
1. Fix or encourage discharge in the hours where $x_t^\star=1$ (e.g., bounds or a deviation penalty $\sum_t |d_t-d_t^{(0)}|$).
2. Solve the resulting LP/MILP to obtain a feasible $(c_t,d_t,E_t)$.
3. Use that solution as the warm start / incumbent for the full MILP.

Pipeline summary:
$\text{MILP} \Rightarrow \text{QUBO skeleton (24 qubits)} \Rightarrow \text{LP/MILP projection} \Rightarrow \text{full MILP}$.


In [10]:
import pandas as pd, numpy as np

# Load dataset
df = pd.read_csv("../data/input_data.csv")
p = df["price"].to_numpy()

# Parameters used for the 24-qubit QUBO
A = 1000.0
gamma = 100.0
K = 8

def build_qubo_upper(p, A=1000.0, gamma=100.0, K=8):
    p=np.asarray(p,float); n=len(p)
    Q=np.zeros((n,n),float)
    # revenue (discharge at 4 MWh when x_t=1)
    for t in range(n):
        Q[t,t] += -4.0*p[t]
    # switching penalty gamma * sum_{t=2..24} (x_t - x_{t-1})^2
    for t in range(n):
        deg = 1.0 if (t==0 or t==n-1) else 2.0
        Q[t,t] += gamma*deg
    for t in range(1,n):
        Q[t-1,t] += -2.0*gamma  # only upper triangle
    # cardinality penalty A*(sum x - K)^2, constant A*K^2 omitted
    lin=A*(1-2*K)
    for t in range(n):
        Q[t,t] += lin
    for i in range(n):
        for j in range(i+1,n):
            Q[i,j] += 2.0*A
    return Q

def upper_to_symmetric(Q_upper):
    Q_sym = np.array(Q_upper, float)
    n=Q_sym.shape[0]
    for i in range(n):
        for j in range(i+1,n):
            q=Q_sym[i,j]
            Q_sym[i,j]=q/2.0
            Q_sym[j,i]=q/2.0
    return Q_sym

Q_upper = build_qubo_upper(p, A=A, gamma=gamma, K=K)
Q_sym = upper_to_symmetric(Q_upper)

# Solve exactly (dynamic programming over time and number of 1s; exact for this special structure)
def solve_chain_with_count(p, gamma, m):
    T=len(p)
    C = -2.0*gamma
    L = np.zeros(T)
    for t in range(T):
        deg = 1.0 if (t==0 or t==T-1) else 2.0
        L[t] = -4.0*p[t] + gamma*deg
    
    INF=1e30
    dp = np.full((T, m+1, 2), INF)
    prev = np.full((T, m+1, 2), -1, dtype=int)
    for s in (0,1):
        k=s
        if k<=m:
            dp[0,k,s]=L[0]*s
            prev[0,k,s]=-1
    for t in range(1,T):
        for k in range(m+1):
            for s in (0,1):
                if k - s < 0:
                    continue
                best=INF; best_prev=-1
                for sp in (0,1):
                    val = dp[t-1, k-s, sp]
                    if val>=INF/2: 
                        continue
                    val2 = val + L[t]*s + C*sp*s
                    if val2 < best:
                        best = val2
                        best_prev = sp
                dp[t,k,s]=best
                prev[t,k,s]=best_prev
    s_final = int(dp[T-1,m,1] < dp[T-1,m,0])
    Emin = dp[T-1,m,s_final]
    x = np.zeros(T,dtype=int)
    k=m; s=s_final
    for t in range(T-1,-1,-1):
        x[t]=s
        sp=prev[t,k,s]
        k-=s
        s=sp if sp!=-1 else 0
    return Emin, x

def solve_qubo_skeleton(p, A, gamma, K):
    T=len(p)
    best_E=1e30
    best_x=None
    best_m=None
    for m in range(T+1):
        chain_E, x = solve_chain_with_count(p, gamma, m)
        E = chain_E + A*(m-K)**2
        if E < best_E:
            best_E=E; best_x=x; best_m=m
    return best_E, best_m, best_x

E_opt, m_opt, x_opt = solve_qubo_skeleton(p, A=A, gamma=gamma, K=K)
d0 = 4.0*x_opt

# Diagonalization (eigendecomposition) of the symmetric form
eigvals, eigvecs = np.linalg.eigh(Q_sym)

# Save artifacts
Q_upper_df = pd.DataFrame(Q_upper, index=[f"h{t}" for t in range(1,25)], columns=[f"h{t}" for t in range(1,25)])
Q_sym_df   = pd.DataFrame(Q_sym,   index=[f"h{t}" for t in range(1,25)], columns=[f"h{t}" for t in range(1,25)])

sol_df = df[["hour","price"]].copy()
sol_df["x_discharge"] = x_opt
sol_df["d0_MWh"] = d0

Q_upper_path = "../data/qubo_matrix_upper.csv"
Q_sym_path   = "../data/qubo_matrix_symmetric.csv"
sol_path     = "../data/qubo_solution.csv"

Q_upper_df.to_csv(Q_upper_path, float_format="%.6f")
Q_sym_df.to_csv(Q_sym_path, float_format="%.6f")
sol_df.to_csv(sol_path, index=False)

# Display small summaries
summary = pd.DataFrame({
    "parameter": ["A", "gamma", "K", "m_opt (sum x)", "E_opt (includes A*(m-K)^2)", "switches", "approx_discharge_revenue_EUR"],
    "value": [A, gamma, K, int(m_opt), float(E_opt), int(np.sum(x_opt[1:] != x_opt[:-1])), float(np.sum(4*p*x_opt))]
})

print("QUBO parameter summary", summary)
print("QUBO solution (x and d0 per hour)", sol_df)

eig_summary = pd.DataFrame({
    "eigval": eigvals
})
print("Eigenvalues of Q (symmetric form)", eig_summary.head(10))



QUBO parameter summary                       parameter    value
0                             A  1000.00
1                         gamma   100.00
2                             K     8.00
3                 m_opt (sum x)     8.00
4    E_opt (includes A*(m-K)^2) -3681.96
5                      switches     3.00
6  approx_discharge_revenue_EUR  3981.96
QUBO solution (x and d0 per hour)     hour   price  x_discharge  d0_MWh
0      1   88.96            0     0.0
1      2   83.82            0     0.0
2      3   83.00            0     0.0
3      4   82.56            0     0.0
4      5   82.82            0     0.0
5      6   86.01            0     0.0
6      7  103.21            0     0.0
7      8  133.09            1     4.0
8      9  113.73            1     4.0
9     10   70.89            0     0.0
10    11   34.14            0     0.0
11    12   19.97            0     0.0
12    13   16.49            0     0.0
13    14   13.72            0     0.0
14    15   18.62            0     0.0
15    1

In [11]:
sol_df

,hour,price,x_discharge,d0_MWh
0,1,88.96,0,0.0
1,2,83.82,0,0.0
2,3,83.00,0,0.0
3,4,82.56,0,0.0
4,5,82.82,0,0.0
5,6,86.01,0,0.0
6,7,103.21,0,0.0
7,8,133.09,1,4.0
8,9,113.73,1,4.0
9,10,70.89,0,0.0
